In [561]:
import numpy as np
from scipy.special import gamma
import matplotlib.pyplot as plt
import random
import scipy
%matplotlib tk

In [620]:
# Loading in the data
data = np.loadtxt('MineRecord_teamC.txt',dtype=int)

In [601]:
lambda1(50,data)

3.4267091955395736

## Part B

In [670]:
def lambda1(n0,data):
    '''
    Picking out a random lambda 1 value from the Gamma(a1,b1)
    '''
    # Calculating a & b values
    x = data[:n0]
    a = 3 + np.sum(x)
    b = 1/(1 + n0)
    
    lambda1 = np.random.gamma(a,b)
    return lambda1

def lambda2(n0,data):
    '''
    Picking out a random lambda 2 from Gamma(a2,b2)
    '''
    # Calculating a & b values
    x = data[n0:]
    a = 3 + np.sum(x)
    b = 1/(1 + (112-n0))
    
    lambda2 = np.random.gamma(a,b)
    return lambda2

def poisson(x,Lambda):
    '''
    Calculate the probability of data using a Poisson Distribution, given an intensity value
    tau = 1
    '''
    intensity = (Lambda**x/scipy.special.factorial(x))*np.exp(-Lambda)
    return intensity

def lambdaProb(Lambda,a,b):
    prob = (1/gamma(a))*(b**a)*Lambda**(a-1)*np.exp(-b*Lambda)
    return prob

def n0prob(n0,data):
    '''
    Generates a new n0 value given the data and n0 value
    '''
    # splits the data up
    x1 = data[:int(n0)]
    x2 = data[int(n0):]
    
    # Creates list for the two different products
    product_1 = []
    product_2 = []
    
    # Calculating a and b values
    a1 = 3 + np.sum(x1)
    a2 = 3 + np.sum(x2)
    b1 = 1 + n0
    b2 = 1 + (112-n0)
    
    # Gets lambda values
    l1 = lambda1(n0,data)
    l2 = lambda2(n0,data)
    
    # Calculates the two products
    for i in x1:
        product_1.append(poisson(i,l1))
    for i in x2:
        product_2.append(poisson(i,l2))
    product_1 = np.prod(np.array(product_1))
    product_2 = np.prod(np.array(product_2))

    # Creates a probability distribution
    prob_vals = []
    years = np.array(np.arange(1,113))
    for i in years:
        prob_vals.append(product_1*product_2*l1*l2*1/112)
    prob_vals = np.array(prob_vals)
    
    # Picks a random number
    new_n0 = random.choices(years,weights=prob_vals)
    prob = prob_vals[int(np.array(new_n0))-1]
    
    return np.array(new_n0),prob,l1,l2
    
    

In [672]:
def mcmc(n0_int,data,steps=100):
    '''
    MCMC to figure out the year regulations changed using Gibbs Sampling
    
    Inputs
    ------
    Intial year n0
    Data
    Number of steps (default = 100)    
    
    Algorithm
    ---------
    From n0 picks lambda 1 and 2
    then using those values
    calculates the probability of n0
    choose another n0 value
    repeats 
    ''' 
    n0_vals = []
    n0_prob = []
    l1_vals = []
    l2_vals = []
    n0 = n0_int
    
    for i in range(steps):
        # Calculating probability of n0 being the year
        n0,prob,l1,l2 = n0prob(n0,data)
            
        # appending values
        n0_vals.append(n0)
        n0_prob.append(prob)
        l1_vals.append(l1)
        l2_vals.append(l2)
        
        # Setting new n0 value
        n0 = int(n0)
        
    # Changing into numpy arrays
    n0_vals = np.array(n0_vals)
    n0_prob = np.array(n0_prob)
    l1_vals = np.array(l1_vals)
    l2_vals = np.array(l2_vals)
    
    # Plotting data
    years = np.arange(1851,1963)
    d = plt.figure(1)
    plt.plot(years,data,marker='o',color='black')
    plt.xlabel('Year')
    plt.ylabel('Number of Accidents')
    d.show()
    
    # Plotting the histogram
    h = plt.figure(2)
    n0_vals = n0_vals + 1739 # Converting from int values to years
    plt.hist(n0_vals,color='black',histtype='step',bins=100,weights=n0_prob,density=True)
    h.show()
    
    # Plotting lambda 1 and lambda 2
    l = plt.figure(3)
    plt.scatter(l1_vals,l2_vals,marker='o',color='black',s=.5)
    plt.xlabel('$\lambda 1$')
    plt.ylabel('$\lambda 2$')
    plt.show()
    
    
        
    

In [673]:
mcmc(45,data,steps=10**5)